In [1]:
import lumipy as lm
import time
from lusidjam import RefreshingToken as rt

# Tutorial 1: Client

Lumipy provides a client class for interfacing to the Luminesce web API and getting results of queries back as pandas DataFrames. Any query you would normally send with the CLI or web GUI will work in the same way here when you pass in a query string. The methods on the client match those on the web api with a couple of additions for convenience.  

## Authentication

Credentials can be supplied as a secrets file or as environment variables (for example, when working in a containerised job) or by supplying a valid token directly (e.g. `RefreshingToken()` in Jupyterhub).

The secrets file is expected to have the following form
```
{
    "api" : {
      "tokenUrl": "<auth token url>",
      "username": "<user name>",
      "password": "<user password>",
      "clientId": "<client id>",
      "clientSecret": "<client secret>",
      "lumiApiUrl": "https://<client>.lusid.com/honeycomb/api/"
    }
}
```

## Build the Client Object

If no path or token is supplied to the constructor the client will fall back on trying to find authentication information in the available environment variables. The names of these environment variables correspond to the following secrets fields.

```
    'lumiApiUrl' = 'FBN_LUMI_API_URL',
    'tokenUrl' = 'FBN_TOKEN_URL',
    'username' = 'FBN_USERNAME',
    'password' = 'FBN_PASSWORD',
    'clientId' = 'FBN_CLIENT_ID',
    'clientSecret' = 'FBN_CLIENT_SECRET'
```

In [2]:
client = lm.get_client(token=rt())

## Sending Queries to Luminesce

The client can be used to send SQL query strings to luminesce either synchronously or asynchronously. 

### Define Query in String
In this case we get five rows of instrument data from Lusid. The `^` in Luminesce SQL denotes the subset of columns that are marked as main. Using `*` will get all columns. 

In [3]:
qry_str = """
select 
    ^ 
from 
    Lusid.Instrument
limit 
    5
"""

### Run Synchronously

In [4]:
df = client.query_and_fetch(qry_str)
df

,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,CCY_XDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Special drawing rights,True
1,LUID_007IMHTU,3636352-DESSE-EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.85% BD REDEEM 24/06/2015 EUR 1000,True
2,LUID_0043FC8Z,5224761-CABND-CAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZC NT REDEEM 04/08/2022 CAD 100,True
3,CCY_SBD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solomon Islands dollar,True
4,LUID_0079PN8M,2179045-DESSE-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO PUT (Intercell AG) E 01/01/9999 EUR 21.4413,True


### Run Asynchronously

Start the query and get store the execution ID you get back.

In [5]:
ex_id = client.start_query(qry_str)
print(f'Query started with execution id = {ex_id}')

Query started with execution id = 8100a47d-cbf5-4eef-8c27-29ba03c3b207


Check the query's status. This returns the JSON formatted status response as a dictionary.

In [6]:
status = client.get_status(ex_id)
while not status['status'] == 'RanToCompletion':
    status = client.get_status(ex_id)
    time.sleep(1)

Get the data from the completed query using the execution ID. 

In [7]:
df = client.get_result(ex_id)
df

Fetching data... 📡
  Page   0 downloaded (5 rows).
Data fetch finished


,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,LUID_007LKSRP,4534914-HKSEHK-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO PUT (HSI) E 27/11/2015 HKD 27050,True
1,LUID_0076I1DQ,5263121-USTRCE-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.25% BD REDEEM 13/09/2020 USD 200000 - Reg S,True
2,LUID_007VOPP5,16341-ROBSE-RON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary Shares,True
3,LUID_005AKPYR,5585374-TWTSE-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW CALL (Yuanta Sec Inv Trst) A 25/06/2019 TWD...,True
4,CCY_BZD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Belize dollar,True


## Table and Field Catalog
Returns a table of information on available providers and their fields. 

In [ ]:
client.table_field_catalog()

## History
Query for information on queries that have been run previously

In [9]:
ex_id = client.start_history_query()
print(ex_id)

1b1112e8-1c6e-41d7-820f-84ae0c218b95


In [10]:
status = client.get_status(ex_id)
while not status['status'] == 'RanToCompletion':
    status = client.get_status(ex_id)
    time.sleep(1)

In [ ]:
client.get_history_result(ex_id)